# Prototype

In [1]:
# Imports
import pandas as pd
from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py
import datetime
import json
import math

import schedule
import time
from datetime import datetime, timedelta
from joblib import dump, load

import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import clear_output

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)
sleep = 1

In [2]:
btc = client.get_asset_balance(asset='USDT')
btc["free"]

'6.13000000'

In [21]:
def get_margin_asset(name:str):
    """returns a dictionary with:
    - asset name
    - free
    - locked
    - borrowed
    - interest
    - netAsset"""
    return list(filter(lambda x: x['asset'] == name, client.get_margin_account()["userAssets"]))[0]

def binance_ceil(x:float):
    """returns the ceil to 5 decimal places (to payback borrowed amounts)"""
    return math.ceil(x*1.001*100000)/100000

In [4]:
binance_ceil(3.1111101)

3.11112

In [24]:
get_margin_asset("USDT")

{'asset': 'USDT',
 'free': '29.94558741',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '29.94558741'}

In [25]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.00000367',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '0.00000367'}

In [26]:
transaction = client.create_margin_loan(asset='ZEC', amount='0.15')
order = client.create_margin_order(
    symbol='ZECUSDT',
    side=SIDE_SELL,
    type=ORDER_TYPE_MARKET,
    quantity=0.15)

In [27]:
get_margin_asset("USDT")

{'asset': 'USDT',
 'free': '44.84067741',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '44.84067741'}

In [28]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.00000367',
 'locked': '0',
 'borrowed': '0.15',
 'interest': '0.00000188',
 'netAsset': '-0.14999821'}

In [29]:
q = abs(float(get_margin_asset("ZEC")["netAsset"]))
q

0.14999821

In [32]:
q = binance_ceil(q)
q

0.15015

In [33]:
order1 = client.create_margin_order(
    symbol='ZECUSDT',
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
    quantity=q)

In [34]:
get_margin_asset("USDT")

{'asset': 'USDT',
 'free': '29.90075241',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '29.90075241'}

In [35]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.15000352',
 'locked': '0',
 'borrowed': '0.15',
 'interest': '0.00000188',
 'netAsset': '0.00000164'}

In [36]:
q = abs(float(get_margin_asset("ZEC")["free"]))
q

0.15000352

In [37]:
transaction1 = client.repay_margin_loan(asset='ZEC', amount=str(q))

In [38]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.00003167',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '0.00003167'}

In [9]:
float(client.get_recent_trades(symbol='XMRUSDT', limit=1)[0]["price"])

196.2

In [4]:
def printer():
    """main printer. Fetches latest minutely data, adds features, predicts, buys/sells if necessary"""
    global sleep
    sleep = 1
    busdd = client.get_asset_balance(asset='BUSD')
    busd = float(busdd["free"]) + float(busdd["locked"])
    btcd = client.get_asset_balance(asset='BTC')
    btc_free = float(btcd["free"])
    btc_locked = float(btcd["locked"])
    btc_price = float(client.get_recent_trades(symbol='BTCBUSD', limit=1)[0]["price"])
    clear_output()
    if btc_free<(5/btc_price) and btc_locked>(5/btc_price): #Can just do == 0.
        #BTC already properly stoplossed
        send_message("waiting", str(btc_price))
    elif btc_free>(5/btc_price): #BTC Needing to be stoplossed with oco
        order = set_oco(tp, sl, btc_price)
    else: #If not holding any BTC, see if it's time to buy
        # Download the minutely data (minimum 5000 minutes or so)
        cancel_all_orders()
        df = get_minutely_data(symbol="BTCUSDT", 
                       kline_interval=Client.KLINE_INTERVAL_1MINUTE, 
                       days=3.5)
        # Add moving average
        df["sma"] = features.get_moving_average(df.close, 5000)
        
        if btc_price > df.sma.iloc[-1]: # If price is above moving average
            df = features.add_all_features(df) 
            df.dropna(inplace=True)
            indicators = list(df.columns)[13:]
            df = df[list(indicators)].copy().astype(np.float32)
            result = model.predict(df)[-1]
            if result == 1:
                order = buy(busd)
            else:
                send_message("ml_no", btc_price)
        else: #BTC is below moving average
            send_message("belowma", btc_price, df.sma.iloc[-1])
            sleep = int(df.sma.iloc[-1]-btc_price)
    reset_client()

In [10]:
def reset_client():
    """resets the client to prevent 'read operation timed out'"""
    global client
    global api_key
    global api_secret
    client = Client(api_key=api_key, api_secret=api_secret)

def set_oco(tp, sl, price):
    """sets oco if doesn't already exist"""
    btc = client.get_asset_balance(asset='BTC')["free"]
    order = client.create_oco_order(
        symbol='BTCBUSD',
        side=SIDE_SELL,
        stopLimitTimeInForce=TIME_IN_FORCE_GTC,
        quantity=btc,
        stopPrice=str(round(price*(1-sl/100), 2)+10), 
        stopLimitPrice=str(round(price*(1-sl/100), 2)), #What if +1000 and much lower
        price=str(round(price*(1+tp/100), 2)))
    send_message("oco", str(round(price*(1+tp/100), 2)), str(round(price*(1-sl/100), 2)))
    return order
    
def buy(busd):
    """market limit order to buy with 50% of busd"""
    btc_price = float(client.get_recent_trades(symbol='BTCBUSD', limit=1)[0]["price"])
    order = client.order_limit_buy(
        symbol='BTCBUSD',
        quantity=round(busd*0.5/btc_price, 5),
        price=str(round(btc_price-10, 2)))
    send_message("bought", str(round(btc_price-10, 2)))
    return order
    
def cancel_all_orders():
    """cancels all orders--cancels previous limit buy order if it exists"""
    orders = client.get_open_orders(symbol='BTCBUSD')
    for order in orders:
        result = client.cancel_order(
            symbol='BTCBUSD',
            orderId=order["orderId"])
        
def get_minutely_data(symbol:str, kline_interval:object, start="1 Jan 1900", days=3.5):
    """
    downloads binance data
    symbol: BTCBUSD
    kline_interval: Client.KLINE_INTERVAL_1DAY, Client.KLINE_INTERVAL_1DAY, Client.KLINE_INTERVAL_1DAY
    interval_name: only used for csv name: BTCUSDT-interval_name.csv
    start: empty if from the very beginning
    """
    d = datetime.today() - timedelta(days=days)
    start_date = d.strftime("%d %b %Y %H:%M:%S")
    today = datetime.today().strftime("%d %b %Y %H:%M:%S")
    klines = client.get_historical_klines(symbol, kline_interval, start_date, today, 1000)
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)
    data_latest = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    
    index = data.index[(data['open'] == data_latest.iloc[0].open) & (data['high'] == data_latest.iloc[0].high)].tolist()[0]
    result = pd.concat([data[:index], data_latest], ignore_index=True, sort=False)
    return result

In [115]:
def get_z_score(hr = 1.534477217835591): # Warning, sometimes it gives nan, just rerun (binance's fault)
    '''gets the latest z-score, given hedge ratio hr'''
    zec = get_minutely_data("ZECUSDT", Client.KLINE_INTERVAL_1MINUTE, days=1.2)
    xmr = get_minutely_data("XMRUSDT", Client.KLINE_INTERVAL_1MINUTE, days=1.2)
    df = pd.to_numeric(zec.open.rename("A")).to_frame()
    df["B"] = pd.to_numeric(xmr.open)
    spread = pd.Series((df['B'] - hr * df['A'])).rename("spread").to_frame()
    spread["mean"] = spread.spread.rolling(2000).mean()
    spread["std"] =  spread.spread.rolling(2000).std()
    spread["zscore"] = pd.Series((spread["spread"]-spread["mean"])/spread["std"])
    return spread.iloc[-1].zscore

In [117]:
float(client.get_recent_trades(symbol='ZECUSDT', limit=1)[0]["price"])

101.04

In [6]:
# df = get_minutely_data(symbol="BTCUSDT", 
#                        kline_interval=Client.KLINE_INTERVAL_1MINUTE, 
#                        days=10)
# df = features.add_all_features(df) 
# df.dropna(inplace=True)
# indicators = list(df.columns)[12:]
# indicators.append('close')
# df = df[list(indicators)].copy().astype(np.float32)

# df.to_csv('../data/debugging.csv', index=False)

In [7]:
# df

In [13]:
schedule.clear()
schedule.every().minute.at(":01").do(printer)

Every 1 minute at 00:00:01 do printer() (last run: [never], next run: 2021-06-14 09:39:01)

In [14]:
# Discord # Saving message
def get_message():
    """gets the message"""
    with open('message.txt') as json_file:
        new_message = json.load(json_file)
        return new_message
def send_message(m:str, a="", b=""):
    global sleep
    data = get_message()
    if m == "waiting":
        data["message"] = f"awaiting tp ${data['tp']} and sl ${data['sl']} orders to fulfill. Currently price ${str(a)} is ${str(round(float(data['tp'])-float(a), 2))} away from tp and ${str(round(float(a)-float(data['sl']), 2))} from stop loss" 
        sleep = int(min(round(float(data['tp'])-float(a), 2), round(float(a)-float(data['sl']), 2)))
        data["wait"] = int(max(round(float(data['tp'])-float(a), 2), round(float(a)-float(data['sl']), 2)))+30
        print("awaiting tp and sl orders to fulfill")
    elif m == "oco":
        data["message"] = f"@everyone Set take profit at ${str(a)} and stop loss at ${str(b)}"
        data["tp"] = str(a)
        data["sl"] = str(b)
        data["wait"] = 60
    elif m == "belowma":
        data["message"] = "Price $" + str(a) + " is below moving average $" + str(round(b, 2)) + " by $" + str(round(b-a, 2))
        data["wait"] = sleep+60
    elif m == "bought":
        data["message"] = f"@everyone Just bought at ${str(a)}"
        data["wait"] = 60
    elif m == "ml_no":
        data["message"] = "ML model says no at current price $" + str(a)
        data["wait"] = 60
    with open('message.txt', 'w') as outfile:
        json.dump(data, outfile)

In [15]:
sleep = 1
while True:
    clear_output()
    schedule.run_pending()
    print(f"sleeping for {sleep} seconds")
    time.sleep(sleep)

awaiting tp and sl orders to fulfill
sleeping for 614 seconds


KeyboardInterrupt: 